🔹 1. Nguyên lý hoạt động (không công thức nặng!)
🎯 Mục tiêu:
Mô phỏng cách não người xử lý thông tin — bằng cách kết nối nhiều "nơ-ron nhân tạo" thành mạng, để học biểu diễn phức tạp từ dữ liệu.

🧠 Ý tưởng chính — “Học từng lớp, từ đơn giản đến phức tạp”
Nơ-ron đơn lẻ ≈ Logistic Regression:
→ Nhận đầu vào → kết hợp có trọng số → đưa qua hàm kích hoạt (sigmoid, ReLU...) → ra kết quả.
Mạng neural = nhiều lớp nơ-ron xếp chồng:
Lớp đầu vào (input): dữ liệu thô (pixel, từ, số...).
Lớp ẩn (hidden layers): học đặc trưng ngày càng trừu tượng.
Ví dụ ảnh mèo:
Lớp 1: phát hiện cạnh, góc
Lớp 2: phát hiện mắt, tai
Lớp 3: phát hiện “mặt mèo”
Lớp đầu ra (output): dự đoán (xác suất các lớp).
🔍 Làm sao mạng "học"?
Forward pass: đưa dữ liệu qua mạng → ra dự đoán.
So sánh dự đoán với nhãn thật → tính lỗi (loss).
Backpropagation: lan truyền lỗi ngược lại, điều chỉnh trọng số để giảm lỗi.
Lặp lại hàng nghìn lần → mạng tự cải thiện.
🌟 Không cần nhớ đạo hàm!
Chỉ cần hiểu: mạng thử → sai → sửa → thử lại → ngày càng đúng hơn. 

✅ Ưu điểm:
Rất mạnh với dữ liệu phức tạp: ảnh, âm thanh, văn bản.
Tự động học đặc trưng — không cần thiết kế tay.
Linh hoạt: có thể mở rộng thành CNN, RNN, Transformer...
❌ Hạn chế:
Cần nhiều dữ liệu để học tốt.
Tốn tài nguyên (GPU, thời gian).
"Hộp đen": khó giải thích tại sao ra kết quả đó.
Dễ overfit nếu không kiểm soát.
 3. Khi nào dùng Neural Networks?
Dữ liệu
rất lớn
(hàng chục nghìn mẫu trở lên)
Dữ liệu
rất nhỏ
(< 1000 mẫu) → dùng RF/SVM thay
Dữ liệu
phi cấu trúc
: ảnh, âm thanh, văn bản
Dữ liệu
bảng đơn giản
(Excel-style) → RF thường đủ
Cần
độ chính xác cực cao
(computer vision, NLP)
Cần
giải thích mô hình
(y tế, tài chính)
Có
GPU/tài nguyên tính toán
Chỉ có CPU và thời gian hạn chế

In [1]:
import numpy as np

class SimpleNeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.1):
        self.lr = learning_rate
        # Khởi tạo trọng số ngẫu nhiên
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01
        self.b2 = np.zeros((1, output_size))

    def _relu(self, x):
        return np.maximum(0, x)

    def _softmax(self, x):
        # Tránh overflow
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def forward(self, X):
        # Lớp ẩn
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self._relu(self.z1)
        # Lớp đầu ra
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self._softmax(self.z2)
        return self.a2

    def backward(self, X, y_true, y_pred):
        m = X.shape[0]

        # Gradient tại lớp đầu ra
        dz2 = y_pred - y_true  # với softmax + cross-entropy
        dW2 = np.dot(self.a1.T, dz2) / m
        db2 = np.sum(dz2, axis=0, keepdims=True) / m

        # Gradient tại lớp ẩn
        da1 = np.dot(dz2, self.W2.T)
        dz1 = da1 * (self.z1 > 0)  # đạo hàm của ReLU
        dW1 = np.dot(X.T, dz1) / m
        db1 = np.sum(dz1, axis=0, keepdims=True) / m

        # Cập nhật trọng số
        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2
        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1

    def fit(self, X, y, epochs=1000):
        # Chuyển y thành one-hot encoding
        n_classes = y.max() + 1
        y_onehot = np.eye(n_classes)[y]

        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss = -np.mean(np.sum(y_onehot * np.log(y_pred + 1e-9), axis=1))
            self.backward(X, y_onehot, y_pred)
            if epoch % 200 == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

    def predict(self, X):
        y_pred = self.forward(X)
        return np.argmax(y_pred, axis=1)

# --- Thử nghiệm ---
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Chuẩn hóa (giúp mạng học ổn định hơn)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Huấn luyện mạng neural tự code
nn_model = SimpleNeuralNetwork(
    input_size=4, 
    hidden_size=8, 
    output_size=3, 
    learning_rate=0.1
)
nn_model.fit(X_train, y_train, epochs=1000)

# Dự đoán
y_pred = nn_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"\n✅ Độ chính xác (Neural Network tự code): {acc:.2%}")

# So sánh với sklearn MLPClassifier
from sklearn.neural_network import MLPClassifier
sk_nn = MLPClassifier(hidden_layer_sizes=(8,), max_iter=1000, random_state=42)
sk_nn.fit(X_train, y_train)
sk_acc = sk_nn.score(X_test, y_test)
print(f"🔍 Sklearn MLP độ chính xác: {sk_acc:.2%}")

Epoch 0, Loss: 1.0986
Epoch 200, Loss: 0.2604
Epoch 400, Loss: 0.1234
Epoch 600, Loss: 0.0873
Epoch 800, Loss: 0.0736

✅ Độ chính xác (Neural Network tự code): 100.00%
🔍 Sklearn MLP độ chính xác: 97.78%


c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
